# Projet 07 : Effectuer une prédiction de revenus

---

## <font color=blue>Sommaire</font>

[1. Exploration des données utilisées (notebook séparé)](data_analyst_projet07_M1_M2_analyse.ipynb)  
  
[2. Concentration des revenus (notebook séparé)](data_analyst_projet07_M1_M2_analyse.ipynb)  
  
[3. Echantillonnage des individus (notebook séparé)](data_analyst_projet07_M3_echantillonnage.ipynb)  
  
[4. Modélisation du revenu des individus](#4_0)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[4.1. Préparation des données](#4_0)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[4.2. ANOVA du revenu basée sur le pays de l'individu](#4_2)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[4.3. ANOVA du logarithme du revenu basée sur le pays de l'individu](#4_3)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[4.4. ANOVA du revenu basée sur le revenu moyen et l'indice de Gini du pays de l'individu](#4_4)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[4.5. ANOVA du logarithme du revenu basée sur le logarithme du revenu moyen et l'indice de Gini du pays](#4_5)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[4.6. ANOVA du revenu basée sur le revenu moyen et l'indice de Gini du pays, et sur la classe de revenu des parents](#4_6)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[4.7. ANOVA du logarithme du revenu basée sur le logarithme du revenu moyen et l'indice de Gini du pays, et sur la classe de revenu des parents](#4_7)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[4.8. ANOVA du logarithme du revenu basée sur le logarithme du revenu moyen et l'indice de Gini du pays, et sur le logarithme du revenu des parents](#4_8)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[4.9. Conclusion de l'étude](#4_9)

---
**Importation des librairies**

In [2]:
import numpy as np
import pandas as pd
from sklearn import model_selection

<a id='4_0'></a>

---
## <font color=blue> Mission 4 : Modélisation du revenu des individus </font>

**Note importante**

Les analyses détaillées, avec les graphiques liés à ces analyses, sont disponibles dans les notebooks mis en lien dans chaque résumé des résultats obtenus lors de ces analyses.

### 4.1. Préparation des données

Cette partie permet de sauvegarder dans des fichiers csv distincts les seules variables nécessaires pour chaque analyse.  
L'intérêt, vu le grand nombre de données, est de charger à chaque fois le fichier le plus léger possible pour ne pas surcharger inutilement la mémoire du système. Cette approche permet également de calculer les logarithmes des revenus en amont de l'analyse des modèles, toujours dans un but d'alégement du calcul des modèles.

In [3]:
individus = pd.read_csv("save_echantillon_individus.csv", index_col=0)
individus.head()

C:\Anaconda3\envs\Jupyter\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,country_code,child_income,parent_centile,revenu_moyen,gini_from_centiles
0,ALB,728.89795,1,2994.829902,32.141019
1,ALB,728.89795,1,2994.829902,32.141019
2,ALB,728.89795,1,2994.829902,32.141019
3,ALB,728.89795,1,2994.829902,32.141019
4,ALB,728.89795,1,2994.829902,32.141019


In [4]:
individus.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5642420 entries, 0 to 5642419
Data columns (total 5 columns):
country_code          object
child_income          float64
parent_centile        int64
revenu_moyen          float64
gini_from_centiles    float64
dtypes: float64(3), int64(1), object(1)
memory usage: 258.3+ MB


In [5]:
centiles = pd.read_csv("save_centiles_201909.csv", index_col=0)
centiles.drop(columns=["year","gdp","country"], inplace=True)
centiles.set_index(["country_code","centile"], inplace=True)
centiles.columns = ["parent_income"]
centiles.head()

parent_income
country_code centile               
ALB          1            728.89795
             2            916.66235
             3           1010.91600
             4           1086.90780
             5           1132.69970

In [6]:
individus = individus.merge(centiles, how='left', left_on=["country_code","parent_centile"], right_index=True)
individus["ln_child_income"] = np.log(individus.child_income)
individus["ln_revenu_moyen"] = np.log(individus.revenu_moyen)
individus["ln_parent_income"] = np.log(individus.parent_income)
individus.head()

,country_code,child_income,parent_centile,revenu_moyen,gini_from_centiles,parent_income,ln_child_income,ln_revenu_moyen,ln_parent_income
0,ALB,728.89795,1,2994.829902,32.141019,728.89795,6.591534,8.004643,6.591534
1,ALB,728.89795,1,2994.829902,32.141019,728.89795,6.591534,8.004643,6.591534
2,ALB,728.89795,1,2994.829902,32.141019,728.89795,6.591534,8.004643,6.591534
3,ALB,728.89795,1,2994.829902,32.141019,728.89795,6.591534,8.004643,6.591534
4,ALB,728.89795,1,2994.829902,32.141019,728.89795,6.591534,8.004643,6.591534


In [7]:
analyse01 = individus[["child_income","country_code"]].copy()
analyse01.to_csv("data_analyse01.csv")

In [8]:
analyse02 = individus[["ln_child_income","country_code"]].copy()
analyse02.to_csv("data_analyse02.csv")

In [9]:
analyse03 = individus[["child_income","revenu_moyen","gini_from_centiles"]].copy()
analyse03.to_csv("data_analyse03.csv")

In [10]:
analyse04 = individus[["ln_child_income","ln_revenu_moyen","gini_from_centiles"]].copy()
analyse04.to_csv("data_analyse04.csv")

In [11]:
analyse05 = individus[["child_income","revenu_moyen","gini_from_centiles","parent_centile"]].copy()
analyse05.to_csv("data_analyse05.csv")

In [12]:
analyse06 = individus[["ln_child_income","ln_revenu_moyen","gini_from_centiles","parent_centile"]].copy()
analyse06.to_csv("data_analyse06.csv")

In [13]:
analyse07 = individus[["child_income","revenu_moyen","gini_from_centiles","parent_income"]].copy()
analyse07.to_csv("data_analyse07.csv")

In [14]:
analyse08 = individus[["ln_child_income","ln_revenu_moyen","gini_from_centiles","ln_parent_income"]].copy()
analyse08.to_csv("data_analyse08.csv")

In [15]:
elasticite = pd.read_csv("save_elasticite.csv", index_col=0)
elasticite.head()

,country_name,decennie_etudiee,coef_elasticite,region,area
country_code,,,,,
CHN,China,1960.0,0.399000,East Asia & Pacific,Asia
MNG,Mongolia,1970.0,0.401002,East Asia & Pacific,Asia
MYS,Malaysia,1960.0,0.540000,East Asia & Pacific,Asia
TLS,Timor-Leste,1970.0,0.701994,East Asia & Pacific,Asia
VNM,Vietnam,1960.0,0.480000,East Asia & Pacific,Asia


In [16]:
indiv_full = individus.merge(elasticite[["coef_elasticite"]], how='left', left_on="country_code", right_index=True)
indiv_full.head()

,country_code,child_income,parent_centile,revenu_moyen,gini_from_centiles,parent_income,ln_child_income,ln_revenu_moyen,ln_parent_income,coef_elasticite
0,ALB,728.89795,1,2994.829902,32.141019,728.89795,6.591534,8.004643,6.591534,0.815874
1,ALB,728.89795,1,2994.829902,32.141019,728.89795,6.591534,8.004643,6.591534,0.815874
2,ALB,728.89795,1,2994.829902,32.141019,728.89795,6.591534,8.004643,6.591534,0.815874
3,ALB,728.89795,1,2994.829902,32.141019,728.89795,6.591534,8.004643,6.591534,0.815874
4,ALB,728.89795,1,2994.829902,32.141019,728.89795,6.591534,8.004643,6.591534,0.815874


In [17]:
analyse11 = indiv_full[["ln_child_income","ln_revenu_moyen","gini_from_centiles","parent_centile","coef_elasticite"]].copy()
analyse11.to_csv("data_analyse11.csv")

In [18]:
analyse12 = indiv_full[["ln_child_income","ln_revenu_moyen","gini_from_centiles","ln_parent_income","coef_elasticite"]].copy()
analyse12.to_csv("data_analyse12.csv")

**Echantillonage**

Du fait du grand nombre de données, mon système n'arrive pas à calculer les indices VIF pour l'analyse du modèle 06. Par conséquent, je crée un sous-échantillon pour ce calcul. Ce sous-échantillon représente 20% de l'échantillon de référence, ce qui représente tout de même plus d'un million d'observations.  
Afin de garder un sous-échantillon représentatif, j'applique une stratégie de stratification basée sur le code pays et le centile du revenu des parents. Ainsi mon sous-échantillon conserve la même proportion de données par pays et par centile de revenu des parents que l'échantillon original.

In [22]:
echantillon06_train, echantillon06_test = model_selection.train_test_split(analyse06, test_size=0.2, shuffle=True, stratify=individus[["country_code","parent_centile"]])
echantillon06_test.to_csv("data_echantillon06.csv")

<a id='4_2'></a>

### 4.2.  ANOVA du revenu basée sur le pays de l'individu

Dans cette première analyse, je cherche à expliquer le revenu d'un individu par son appartenance à un pays. Le détail des calculs est disponible sur le notebook : [data_analyst_projet07_M41_modelisation01](data_analyst_projet07_M41_modelisation01.ipynb)

Les résultats de ce modèle laissent penser que le revenu des individus dépend de leur pays. Cependant, ce modèle n'est pas satisfaisant car il présente un problème de linéarité et d'hétéroscédasticité. L'analyse des résidus suggère d'utiliser le logarithme du revenu des enfants afin d'obtenir une relation plus linéaire. Je vérifie cette hypothèse avec une deuxième modélisation.

<a id='4_3'></a>

### 4.3. ANOVA du logarithme du revenu basée sur le pays de l'individu

Dans cette deuxième analyse, je cherche toujours à expliquer le revenu d'un individu par son appartenance à un pays. Afin d'avoir une relation plus linéaire, je modélise le logarithme du revenu en fonction de son pays. Le détail des calculs est disponible sur le notebook : [data_analyst_projet07_M42_modelisation02](data_analyst_projet07_M42_modelisation02.ipynb)

L'opération de feature engineering consistant à remplacer le revenu des enfants par le logarithme du revenu des enfants (logarithme népérien) améliore le modèle. La relation est plus linéaire ce qui permet de mieux respecter les hypothèses. La normalité des résidus s'améliore avec une distribution qui devient plus symétrique, même si cette symétrie est encore loin d'être idéale. L'homoscédasticité semble respectée graphiquement. En conséquence, les résultats de la modélisation s'améliorent.  

Avec une seule variable explicative, je n'ai pas de problème de colinéarité. Certaines observations sont influentes mais aucune n'est à la fois influente et atypique.  
  
La part de dispersion expliquée par le modèle (R²) augmente de 49,4% à 72,9%. La variable country_code est considérée comme significative par l'ANOVA. La meilleure qualité du modèle se traduit par une diminution des indicateurs AIC et BIC de 1,155e+08 à 1,231e+07. Ce modèle conclut donc que le revenu d'un individu dépend du pays dans lequel il vit.    
  
Toutefois, ce modèle est complexe, avec un coefficient différent pour chaque pays. Une idée pour simplifier le modèle est de caractériser un pays par son revenu moyen et son indice de Gini au lieu d'utiliser son nom ou son code pays. Ainsi, nous limitons le nombre de coefficients estimés à 2 (en sus de la constante) et les pays avec des caractéristiques similaires auront des estimations similaires.

<a id='4_4'></a>

### 4.4.  ANOVA du revenu basée sur le revenu moyen et l'indice de Gini du pays de l'individu

La troisième analyse modélise le revenu de l'individu en fonction du revenu moyen et l'indice de Gini de son pays. Le détail des calculs est disponible sur le notebook : [data_analyst_projet07_M43_modelisation03](data_analyst_projet07_M43_modelisation03.ipynb)

Le fait de considérer à nouveau la valeur non transformée des revenus de l'individu entraîne le non-respect des trois hypothèses requises. Il est inutile de pousser plus avant l'étude de ce modèle. L'utilisation des valeurs du logarithme des revenus (revenu de l'enfant et revenu moyen du pays) devrait fournir un modèle plus linéaire, donc plus pertinent et exploitable.

<a id='4_5'></a>

### 4.5. ANOVA du logarithme du revenu basée sur le logarithme du revenu moyen et l'indice de Gini du pays

La quatrième analyse, disponible sur le notebook [data_analyst_projet07_M44_modelisation04](data_analyst_projet07_M44_modelisation04.ipynb), modélise le logarithme du revenu d'un individu en fonction de l'indice de Gini et du logarithme du revenu moyen de son pays.

La symétrie des résidus s'améliore nettement avec ce modèle. Nous nous rapprochons de plus en plus d'une distribution normale, même si la droite de Henry montre que les valeurs extrêmes sont difficiles à modéliser linéairement.  
Les graphiques montrent que les hypothèses de linéarité et d'hétéroscédasticité sont bien respectées. Tout au plus, nous pouvons nous questionner sur l'indépendance des résidus par rapport aux valeurs de l'indice de Gini. En effet, la dispersion des résidus semble augmenter très légèrement lorsque l'indice de Gini croît.  
Mais cette constatation est explicable par la définition même de l'indice de Gini. L'indice de Gini définit la concentration des revenus au sein de la population du pays. Plus il est élevé, plus les revenus sont concentrés. Cette concentration accrue se traduit en général par des riches plus riches et des pauvres plus pauvres par rapport à la population médiane du pays. Et donc par une dispersion des revenus plus importante.

Le coefficient de corrélation de Pearson et les indices VIF sont faibles. Ils ne mettent en évidence aucun problème de colinéarité simple ou multiple.  
Le nombre d'observations influentes augmente. Surtout, nous avons désormais quasiment 1% des observations qui sont à la fois atypiques et influentes. Cet aspect sera à investiguer si nous optons au final pour ce modèle.

Les résultats sont très similaires à ceux obtenus avec le modèle basé sur le country_code. La part de la variance expliquée par le modèle (R²) est de 72,9%, comme pour le modèle 2. Nous obtenons également un AIC et un BIC équivalents au modèle 2.
L'ANOVA conclut à la significativité des deux variables. Il faut noter que le logarithme du revenu moyen est le très net contributeur principal du modèle (omega squared de 0,686 contre 0.012 pour l'indice de Gini).

Outre ses résultats satisfaisants et une meilleure symétrie de la distribution des résidus, ce modèle présente deux avantages. D'une part, il est bien plus simple avec seulement deux coefficients estimés (en plus de la constante). D'autre part, si la situation d'un pays évolue, ce modèle devrait mieux prendre en compte cette évolution car il base ses estimations sur les caractéristiques du pays plutôt que sur son nom.  
**Ce modèle permet de dire que le pays de naissance, qu'il soit identifié par son nom ou par la combinaison du revenu moyen et de l'indice de Gini du pays, explique 72,9% de la variance totale. Les autres facteurs non considérés par le modèle expliquent 27,1% de la variance totale.**

Une limitation de ce modèle est de ne prévoir les revenus des enfants qu'en fonction des caractéristiques du pays. Nous pouvons légitimement penser que les revenus des parents influent sur les revenus des enfants. Nous pouvons introduire cette composante à notre analyse à travers la classe (le centile) à laquelle les revenus des parents appartiennent.

<a id='4_6'></a>

### 4.6. ANOVA du revenu basée sur le revenu moyen et l'indice de Gini du pays, et sur la classe de revenu des parents

La cinquième analyse modélise le revenu d'un individu en fonction des caractéristiques du pays (revenu moyen et indice de Gini) et de la classe de revenu des parents. Elle disponible dans le notebook [data_analyst_projet07_M45_modelisation05](data_analyst_projet07_M45_modelisation05.ipynb).  
Nous avons toujours un problème de linéarité du fait de ne pas considérer le logarithme des revenus. De ce fait, ce modèle n'est pas exploitable.

<a id='4_7'></a>

### 4.7. ANOVA du logarithme du revenu basée sur le logarithme du revenu moyen et l'indice de Gini du pays, et sur la classe de revenu des parents

Le sixième modèle estime le logarithme du revenu d'un individu en fonction de l'indice de Gini et du logarithme du revenu moyen de son pays et en fonction de la classe de revenu de ses parents. L'analyse est disponible dans le notebook [data_analyst_projet07_M46_modelisation06](data_analyst_projet07_M46_modelisation06.ipynb).

La part de variance expliquée par le modèle (R²) est la plus haute obtenue jusqu'à maintenant : 78,6% de la dispersion totale. Ce modèle présente également la meilleure qualité prédictive, avec les scores AIC et BIC les plus faibles de tous les modèles testés jusqu'à maintenant : 1,098e+7.  
L'ANOVA conclut à la significativité des trois variables. Le logarithme du revenu moyen du pays reste le principal contributeur du modèle (omega squared = 0.686), loin devant la classe de revenu des parents et l'indice de Gini du pays (omega squared respectifs de 0.064 et 0.012).

Les résidus ne sont pas normaux mais ils présentent une symétrie assez bonne. La droite de Henry confirme la difficulté à modéliser les valeurs extrêmes. Elle marque toutefois une amélioration dans les valeurs extrêmes avec la disparition des effets de seuil au profit d'une meilleure progressivité.

Les graphiques montre une répartition assez homogène en fonction des valeurs de l'endogène et des exogènes. Ils permettent de valider les hypothèses de linéarité de la relation et d'homoscédasticité.

Le coefficient de corrélation de Pearson et les indices VIF sont faibles. Ce modèle ne présente aucun problème de colinéarité simple ou multiple.

L'analyse des observations montre qu'il n'y a pas d'observation considérée comme atypique et qu'il y a un peu moins de 5% des observations considérées comme influentes. Aucun individu n'est à la fois atypique et influent. Il n'y a donc pas de problème de modélisation sur ce point.

En conclusion, l'ajout de la classe de revenu des parents est très bénéfique:
- les 3 variables (logarithme du revenu moyen, indice de Gini et classe de revenu des parents) sont significatives ;
- la part de dispersion expliquée par le modèle augmente ainsi que sa qualité prédictive ;
- il n'y a plus de variable atypique, et par conséquent plus d'obervation à la fois atypique et influente.

**Ce modèle permet de dire que le revenu d'un individu dépend à la fois du pays de naissance et de la classe de revenu de ses parents, qui expliquent 78,6% de la variance totale. Les autres facteurs non considérés par le modèle expliquent 21,4% de la variance totale.**  
Le coefficient de régression associé à l'indice de Gini du pays est légèrement négatif (-0.0165 lorsque le modèle est basé sur des valeurs exogènes quantitatives centrées et réduites). Cela signifie que lorsque l'indice de Gini du pays augmente, le revenu estimé d'un individu diminue toute chose égale par ailleurs. Donc **le fait de vivre dans un pays inégalitaire (avec une grande concentration des revenus et donc un indice de Gini élevé) défavorise plus de personnes qu'il n'en favorise.**

Le seul inconvénient éventuel de ce modèle est son nombre de coefficients. 101 coefficients sont estimés en plus de la constante : un pour le logarithme du revenu moyen du pays, un pour l'indice de Gini du pays et 99 pour les classes de revenu des parents. Est-ce que remplacer la classe de revenu des parents par le logarithme du revenu moyen pour cette classe serait bénéfique ?

<a id='4_8'></a>

### 4.8. ANOVA du logarithme du revenu basée sur le logarithme du revenu moyen et l'indice de Gini du pays, et sur le logarithme du revenu des parents

L'analyse de ce modèle où l'on remplace la classe de revenu des parents par le logarithme du revenu des parents est faite dans le notebook [data_analyst_projet07_M47_modelisation07](data_analyst_projet07_M47_modelisation07.ipynb).  
Cette approche est rejetée car elle introduit un problème de colinéarité simple entre le logarithme du revenu moyen et le logarithme du revenu des parents. De plus, elle se caractérise par des observations à la fois atypiques et influentes, dont est exempt le modèle précédent.

<a id='4_9'></a>

### 4.9. Conclusion de l'étude

Au final, je retiens le modèle qui estime le logarithme du revenu des individus à l'aide du logarithme du revenu moyen du pays, de l'indice de Gini du pays et de la classe de revenu des parents (modèle étudié en [4.7](#4_7)).  
Au delà des bons résultats de l'analyse de cette modélisation, la meilleure modélisation des valeurs extrêmes, le respect des différentes hypothèses, l'absence de colinéarité et l'absence d'observations à la fois atypiques et influentes marquent la supériorité de ce modèle sur les autres modèles étudiés. 